In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from keras._tf_keras.keras.models import Sequential, load_model
from keras._tf_keras.keras.layers import LSTM, Dense, Input
import matplotlib.pyplot as plt
from UsefulFunctions import create_sequences, rolling_forecast, FitLSTM, FitLSTM_1layer

# Task 2.1

In [ ]:
# Load electricity spot price data
df = pd.read_csv("Elspotprices2nd.csv")
df["HourUTC"] = pd.to_datetime(df["HourUTC"])
df.set_index("HourUTC", inplace=True)
df = df.sort_index()

# Split data into training (Jan 2019 to Aug 2024) and testing (Sep 2024)
lstm1_train_data = df.loc[:"2024-08-31"].values.reshape(-1, 1)
lstm1_test_data = df.loc["2024-09-01":"2024-09-30"].values.reshape(-1, 1)

# Normalize both training and testing data
lstm1_scaler = MinMaxScaler()
lstm1_train_scaled = lstm1_scaler.fit_transform(lstm1_train_data)
lstm1_test_scaled = lstm1_scaler.transform(lstm1_test_data)

In [ ]:
# Define input features and hyperparameters
window_size = 24
n_lookahead = 24
n_neurons = 64
n_features = 1
epochs = 10
dropout = 0

# Fit the LSTM model
# The ordered_validation parameter is set to True to ensure that the validation set is ordered in time,
# and not randomly shuffled. This is important for time series data to maintain the temporal order.
lstm2_model = FitLSTM_1layer(lstm1_train_scaled, window_size, n_features, 
                             n_lookahead, n_neurons, epochs, dropout, ordered_validation=True)
lstm2_model.save("lstm1_model_test.keras")

# Generate predictions
lstm1_pred_scaled = rolling_forecast(lstm2_model, lstm1_train_scaled, lstm1_test_scaled, window_size, n_lookahead)

# Inverse transform to get the actual values from the scaled predictions
lstm1_pred_inv = lstm1_scaler.inverse_transform(lstm1_pred_scaled.reshape(-1, 1))

# Forecast with persistence (naive forecast using previous values)
Persistence = np.concatenate((lstm1_train_data[-n_lookahead:], # Start with last <n_lookahead> hours of training data
                              lstm1_test_data[:-n_lookahead]), # Add the test data to the persistence forecast
                              axis=0)  # Combine arrays

# Calculate and print RMSE values for persistence and forecasts
rmse_lstm1 = root_mean_squared_error(lstm1_test_data, lstm1_pred_inv)
rmse_persistence = root_mean_squared_error(Persistence, lstm1_test_data)
print(f"RMSE for LSTM (No exogenous): {rmse_lstm1:.2f}")
print(f"RMSE for Persistence: {rmse_persistence:.2f}")

# Plot the forecasts
plt.figure(figsize=(10, 4), dpi=100)
plt.plot(np.arange(1, len(lstm1_train_data) + 1), lstm1_train_data, color="black", label="Training set")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(lstm1_pred_inv.flatten()) + 1), lstm1_pred_inv.flatten(), color="blue", label="Forecasted values")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(Persistence) + 1), Persistence, color="green", label="Persistence")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(lstm1_test_data.flatten()) + 1), lstm1_test_data.flatten(), color="red", label="Actual values")
plt.legend(loc="upper left")
plt.grid(alpha=0.25)
plt.xlim([len(lstm1_train_data) - 7 * 24, len(lstm1_train_data) + len(lstm1_test_data)])
plt.tight_layout()
plt.show()

### Version with multilayer LSTM
The multilayer setup leads to higher rmse.

In [ ]:
# Define input features and hyperparameters
n_features = 1  # No external input features, only the target variable
n_steps = 24  # The number of time steps the model will look back
n_lookahead = 24  # The number of steps the model will predict ahead
n_neurons = 64  # Number of neurons in the LSTM layer
n_neurons_dense = 36  # Number of neurons in the dense layer
epochs = 10  # Number of training epochs
dropout1 = 0.05  # Dropout rate for the LSTM layer
dropout2 = 0.05  # Dropout rate for the dense layer

# Fit the LSTM model
model = FitLSTM(lstm1_train_scaled, n_steps, n_features, n_lookahead, 
                n_neurons, n_neurons_dense, epochs, dropout1, dropout2)
model.save("The_modelv2.keras")

# Generate predictions
lstm1_pred_scaled = rolling_forecast(model, lstm1_train_scaled, lstm1_test_scaled, n_steps, n_lookahead)

# Inverse transform to get the actual values from the scaled predictions
Forecasts = lstm1_scaler.inverse_transform(lstm1_pred_scaled.reshape(-1, 1))

# Forecast with persistence (naive forecast using previous values)
Persistence = np.concatenate((lstm1_train_data[-n_lookahead:], # Start with last <n_lookahead> hours of training data
                              lstm1_test_data[:-n_lookahead]), # Add the test data to the persistence forecast
                              axis=0)  # Combine arrays

# Calculate and print RMSE values for persistence and forecasts
RMSE_P = root_mean_squared_error(Persistence, lstm1_test_data)
RMSE_F = root_mean_squared_error(Forecasts, lstm1_test_data)
print("RMSE for weekly persistence: ", round(RMSE_P))
print("RMSE for forecasts: ", round(RMSE_F))

# Plot the forecasts
plt.figure(figsize=(10, 4), dpi=100)
plt.plot(np.arange(1, len(lstm1_train_data) + 1), lstm1_train_data, color="black", label="Training set")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(Forecasts) + 1), Forecasts, color="blue", label="Forecasted values")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(Persistence) + 1), Persistence, color="green", label="Persistence")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(lstm1_test_data) + 1), lstm1_test_data, color="red", label="Actual values")
plt.legend(loc="upper left")
plt.grid(alpha=0.25)
plt.xlim([len(lstm1_train_data) - 7 * 24, len(lstm1_train_data) + len(lstm1_test_data)])
plt.tight_layout()
plt.show()

# Task 2.2

In [ ]:
# Load price data
df_prices = pd.read_csv("Elspotprices2nd.csv")
df_prices["HourUTC"] = pd.to_datetime(df_prices["HourUTC"])
df_prices.set_index("HourUTC", inplace=True)
df_prices = df_prices.sort_index()

# Load exogenous data
df_exo = pd.read_csv("ProdConData.csv")
df_exo["HourUTC"] = pd.to_datetime(df_exo["HourUTC"])
df_exo.set_index("HourUTC", inplace=True)
df_exo = df_exo.sort_index()

# Merge datasets
df_combined = df_prices.join(df_exo, how='inner')

# Select target + exogenous features
exogenous_vars = ["GrossConsumptionMWh", "OffshoreWindGe100MW_MWh", "SolarPowerGe40kW_MWh"]
lstm2_features = ["SpotPriceDKK"] + exogenous_vars
df_lstm2 = df_combined[lstm2_features].dropna()

# Train/test split
lstm2_train = df_lstm2.loc["2019-01-01":"2024-08-31"]
lstm2_test = df_lstm2.loc["2024-09-01":"2024-09-30"]

# Normalize
lstm2_scaler = MinMaxScaler()
lstm2_train_scaled = lstm2_scaler.fit_transform(lstm2_train)
lstm2_test_scaled = lstm2_scaler.transform(lstm2_test)

# Create multivariate sequences
def create_sequences_multivariate(data, window_size=24):
    X, y = [], []
    for i in range(len(data) - window_size - 23):
        X.append(data[i:i+window_size])              # Input: 24x4
        y.append(data[i+window_size:i+window_size+24, 0])  # Output: 24 prices
    return np.array(X), np.array(y)

lstm2_window_size = 24
lstm2_X_train, lstm2_y_train = create_sequences_multivariate(lstm2_train_scaled, lstm2_window_size)

# Build model
lstm2_model = Sequential()
lstm2_model.add(LSTM(64, input_shape=(lstm2_window_size, len(lstm2_features))))
lstm2_model.add(Dense(24))  # Predict 24 hours
lstm2_model.compile(loss='mse', optimizer='adam')
lstm2_model.fit(lstm2_X_train, lstm2_y_train, epochs=10, batch_size=32, verbose=1)

# Forecast function
def rolling_forecast_multivariate(model, test_scaled, window_size=24, n_features=4):
    predictions = []
    for day in range(30):
        start_idx = day * 24
        input_seq = test_scaled[start_idx:start_idx+window_size]
        input_seq = input_seq.reshape(1, window_size, n_features)
        pred_scaled = model.predict(input_seq, verbose=0)[0]
        predictions.append(pred_scaled)
    return np.array(predictions)

# Forecast
lstm2_pred_scaled = rolling_forecast_multivariate(
    lstm2_model, lstm2_test_scaled, window_size=lstm2_window_size, n_features=len(lstm2_features)
)

# Inverse transform only price predictions
lstm2_pred_flat = lstm2_pred_scaled.reshape(-1, 1)
dummy = np.zeros((lstm2_pred_flat.shape[0], len(lstm2_features)))
dummy[:, 0] = lstm2_pred_flat[:, 0]
lstm2_pred_inv = lstm2_scaler.inverse_transform(dummy)[:, 0].reshape(30, 24)

# Ground truth
lstm2_true = lstm2_test["SpotPriceDKK"].values.reshape(30, 24)

# Compute RMSE
def compute_rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

rmse_lstm2 = compute_rmse(lstm2_true, lstm2_pred_inv)
print(f"LSTM (with exogenous vars) RMSE: {rmse_lstm2:.2f}")


In [ ]:
"""
Task 2 – Long‑Short‑Term‑Memory (LSTM) day‑ahead forecaster
WITH up to three exogenous variables
——————————————————————————————————————————————
• Target          : DK2 spot price  (column “SpotPriceDKK”)
• Exogenous input : GrossConsumptionMWh,
                    OffshoreWindGe100MW_MWh,
                    SolarPowerGe40kW_MWh
• Forecast horizon: 24 h  (n_lookahead)
• Look‑back window: 48 h  (n_steps)          ← identical to the improved univariate script
• Split           : 2019‑01‑01 – 2024‑08‑31 → train
                    2024‑09‑01 – 2024‑09‑30 → test
• Model           : 2‑layer LSTM from UsefulFunctions.LSTM_multilayer
• Evaluation      : RMSE of LSTM forecast vs. weekly‑persistence benchmark
"""

# ───────────────────────── Imports ──────────────────────────
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential  # only needed if you want to tweak the model further

from UsefulFunctions import LSTM_multilayer        # multi‑layer LSTM builder
# (rolling_forecast from UsefulFunctions is 1‑D; we implement a multivariate analogue below)

# ───────────────────── Helper functions ─────────────────────
def root_mean_squared_error(y_true, y_pred):
    """Wrapper around sklearn’s MSE that returns the square root (RMSE)."""
    return np.sqrt(mean_squared_error(y_true, y_pred))


def create_sequences_multivariate(data: np.ndarray,
                                  window_size: int,
                                  n_lookahead: int):
    """
    Build (X, y) pairs for a *multivariate* sequence‑to‑sequence forecast.
      X shape → [samples, window_size, n_features]
      y shape → [samples, n_lookahead]    (only the *price* column is predicted)
    """
    X, y = [], []
    for i in range(len(data) - window_size - n_lookahead + 1):
        X.append(data[i:i + window_size, :])                 # last 48 h, all features
        y.append(data[i + window_size:i + window_size + n_lookahead, 0])  # next 24 prices
    return np.array(X), np.array(y)


def FitLSTM_multivariate(train_scaled: np.ndarray,
                         n_steps: int,
                         n_features: int,
                         n_lookahead: int,
                         n_neurons: int,
                         n_neurons_dense: int,
                         epochs: int,
                         dropout1: float,
                         dropout2: float):
    """
    Thin wrapper that (i) turns the multivariate windowed data into tensors,
    (ii) calls UsefulFunctions.LSTM_multilayer, (iii) trains the network,
    (iv) plots learning curves, and (v) returns the fitted model.
    """
    # --- 1 ▸ build training tensors
    X_train, y_train = create_sequences_multivariate(
        train_scaled, n_steps, n_lookahead
    )

    # --- 2 ▸ instantiate the model
    model = LSTM_multilayer(
        n_steps,            # same interface as in UsefulFunctions.FitLSTM
        n_features,
        n_neurons,
        n_neurons_dense,
        n_lookahead,
        dropout1,
        dropout2,
    )

    # --- 3 ▸ train
    history = model.fit(
        X_train,
        y_train,
        epochs=epochs,
        validation_split=0.2,
        verbose=1,
    )

    # --- 4 ▸ diagnostics
    plt.figure(figsize=(10, 4))
    plt.plot(history.history["loss"], label="Training loss")
    plt.plot(history.history["val_loss"], label="Validation loss")
    plt.xlabel("Epoch")
    plt.ylabel("MSE")
    plt.title("Learning‑curve (multivariate LSTM)")
    plt.grid(alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

    return model


def rolling_forecast_multivariate(model,
                                  history_scaled: np.ndarray,
                                  test_scaled: np.ndarray,
                                  window_size: int,
                                  n_lookahead: int):
    """
    Multivariate, *rolling* day‑ahead forecast identical in spirit to
    UsefulFunctions.rolling_forecast but supporting n_features > 1.
    """
    predictions = []

    data = np.concatenate((history_scaled, test_scaled), axis=0)
    n_train = len(history_scaled)
    n_test  = len(test_scaled)
    N_days  = int(n_test / n_lookahead)                      # == 30 for September

    for day in range(N_days):
        start = n_train + day * n_lookahead - window_size
        end   = n_train + day * n_lookahead
        input_seq = data[start:end].reshape((1, window_size, data.shape[1]))

        pred_scaled = model.predict(input_seq, verbose=0)[0]  # shape (24,)
        predictions.append(pred_scaled)

    return np.array(predictions)                              # (30, 24)


# ─────────────────────── Data ingestion ─────────────────────
# ➊ price
df_p = pd.read_csv("Elspotprices2nd.csv")
df_p["HourUTC"] = pd.to_datetime(df_p["HourUTC"])
df_p.set_index("HourUTC", inplace=True)
df_p.sort_index(inplace=True)

# ➋ exogenous variables
exo_vars = ["GrossConsumptionMWh",
            "OffshoreWindGe100MW_MWh",
            "SolarPowerGe40kW_MWh"]

df_x = pd.read_csv("ProdConData.csv")
df_x["HourUTC"] = pd.to_datetime(df_x["HourUTC"])
df_x.set_index("HourUTC", inplace=True)
df_x.sort_index(inplace=True)

# ➌ merge – keep only rows where every column is present
df = df_p.join(df_x[exo_vars], how="inner")\
         .rename(columns={"SpotPriceDKK": "Price"})

# ─────────────── Train–test split & scaling ────────────────
train_df = df.loc[:"2024-08-31"]
test_df  = df.loc["2024-09-01":"2024-09-30"]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df)
test_scaled  = scaler.transform(test_df)

# ───────────────────── Hyper‑parameters ─────────────────────
n_features       = train_scaled.shape[1]      # = 4  (price + 3 exo)
n_steps          = 48
n_lookahead      = 24
n_neurons        = 150
n_neurons_dense  = 60
epochs           = 25
dropout1         = 0.10
dropout2         = 0.10

# ─────────────────────── Model training ─────────────────────
model = FitLSTM_multivariate(
    train_scaled,
    n_steps, n_features, n_lookahead,
    n_neurons, n_neurons_dense,
    epochs, dropout1, dropout2
)
model.save("LSTM_Task2_withExo_v2.keras")

# ───────────────────────── Inference ────────────────────────
pred_scaled = rolling_forecast_multivariate(
    model, train_scaled, test_scaled,
    n_steps, n_lookahead
)                                           # shape (30, 24)

# — inverse‑transform ONLY the price column —
flat_pred = pred_scaled.flatten()[:, None]                # (720, 1)
dummy     = np.zeros((flat_pred.shape[0], n_features))
dummy[:, 0] = flat_pred[:, 0]                             # price → first column
price_forecast = scaler.inverse_transform(dummy)[:, 0]\
                     .reshape(pred_scaled.shape)          # (30, 24)

# ─────────────────────── Benchmarks ─────────────────────────
# Weekly persistence: last 24 h of TRAIN + actuals up to t‑24
persistence = np.concatenate(
    (train_df["Price"].values[-n_lookahead:],
     test_df["Price"].values[:-n_lookahead])
)

# For RMSE we need identical shapes
rmse_persistence = root_mean_squared_error(
    test_df["Price"].values, persistence
)

rmse_forecast = root_mean_squared_error(
    test_df["Price"].values, price_forecast.flatten()
)

# ────────────────────────── Plots ───────────────────────────
plt.figure(figsize=(10, 4), dpi=110)
t_train = np.arange(len(train_df))
t_test  = np.arange(len(train_df), len(train_df) + len(test_df))

plt.plot(t_train, train_df["Price"].values,  color="black", label="Training set")
plt.plot(t_test,  test_df["Price"].values,   color="red",   label="Actual price (test)")
plt.plot(t_test,  price_forecast.flatten(), color="blue",  label="LSTM forecast")
# plt.plot(t_test, persistence,              color="green", label="Persistence")

plt.xlim([len(train_df) - 7*24, len(train_df) + len(test_df)])
plt.legend(loc="upper left")
plt.grid(alpha=0.25)
plt.tight_layout()
plt.show()

# ──────────────────────── Results ───────────────────────────
print(f"RMSE – weekly persistence : {rmse_persistence:,.0f} DKK/MWh")
print(f"RMSE – LSTM with exo vars : {rmse_forecast:,.0f} DKK/MWh")